# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [7]:
import os

from openai import OpenAI

client = OpenAI(
    # 中转Key或者直连Key
    api_key=os.getenv('OPENAI_API_KEY'),         
    base_url=os.getenv('BASE_URL') 
)

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一条木栈道，穿过一片郁郁葱葱的草地。栈道延伸向远方，周围是高高的绿色植物，背景中可以看到树木和灌木丛。天空湛蓝，有些云彩点缀，整体给人一种宁静和自然的感觉。这样的景象常常让人感到放松，适合散步或享受大自然的美好。', role='assistant', function_call=None, tool_calls=None))


In [2]:
response.choices[0].message.content

'这幅图呈现了一个宁静美丽的自然景观。图中的主要特点是一条木制的栈道，它穿越在郁郁葱葱的草地上，引向远处看不到的终点。草地上覆盖着茂密的绿色植被，显示出生机勃勃的春夏季节。背景中天空呈现出澄清的蓝色，点缀着几朵轻盈的白云，增添了一种宁静和平和的氛围。\n\n整个场景光线充足，阳光和蓝天与绿色植被的对比令人感觉清新愉快。这样的环境可能是理想的徒步旅行地点，提供了亲近自然和放松心情的完美机会。整体上，这幅图传达了大自然的宁静与和谐，是观赏和体验自然美景的一种'

### 封装成一个函数 query_image_description

In [42]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI(
        # 中转Key或者直连Key
        api_key='sk-wo5EhorUeg5m0eCx24774722BaAc4a06B1E2E80d8d8c1b75',         
        # 代理地址，填写商家中转站或自建OpenAI代理
        # base_url='https://130200.scf.tencentcs.com/v1
        base_url='https://api.xiaoai.plus/v1' 
    )
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [32]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [16]:
pip show openai

Name: openai
Version: 1.30.1
Summary: The official Python library for the openai API
Home-page: 
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: d:\ita\anaconda3\lib\site-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
Note: you may need to restart the kernel to use updated packages.


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [11]:
from openai import OpenAI
import base64
import requests
import json

# client = OpenAI()  # 初始化 OpenAI 客户端
client = OpenAI(
    # 中转Key或者直连Key
    api_key=os.getenv('OPENAI_API_KEY'),         
    base_url=os.getenv('BASE_URL') 
)

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.xiaoai.plus/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [12]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

ConnectTimeout: HTTPSConnectionPool(host='api.openai.com', port=443): Max retries exceeded with url: /v1/chat/completions (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022E6096AD10>, 'Connection to api.openai.com timed out. (connect timeout=None)'))

#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [17]:
content = query_base64_image_description("./images/gpt-4v-2.jpg", "解答一下图像中的第四个问题，并给出详细解释")
print(content)

根据题目中的要求：

1. **小于4的数**：
   - 在1到10的范围内，小于4的数是1, 2, 3。

2. **大于7的数**：
   - 在1到10的范围内，大于7的数是8, 9, 10。

3. **不大于6的数**：
   - 在1到10的范围内，不大于6的数是1, 2, 3, 4, 5, 6。

将这些数字填写到相应的空格中：

- 小于4的数: **1, 2, 3**
- 大于7的数: **8, 9, 10**
- 不大于6的数: **1, 2, 3, 4, 5, 6**

这些填写完成后，确保没有遗漏和错误即可。


In [26]:
content = query_base64_image_description("./images/gpt-4v-1.jpg", "读取图片中的文本，输出源文本，并且在给出一个中文翻译版本")
print(content)

### 原文
My summer holiday  
My summer holiday is coming. I think it will be hot in July and August. So I will not go outside to play with my friends. Every day I will study hard. I am going to get up at 7:00. I am going to do my homework first and then learn about computer. I think my summer holiday will be wonderful.

### 中文翻译
我的暑假  
我的暑假快到了。我认为七月和八月会很热。所以我不会出去和我的朋友们玩。每天我都会努力学习。我打算在七点钟起床。我会先做作业，然后学习计算机。我觉得我的暑假会很美好。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [11]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上手写的文字，内容主要涉及到自然语言处理（NLP）中的训练技术，如Prompt Tuning和LoRA技术。

1. **Prompt Tuning（简要模型调整）**： 提到了使用Prompt Tuning来调整一个小型的Transformer模型。此处解释了输入X（由个别输入X1, X2, ..., Xn组成）。每个输入首先通过一个Embedding过程转换，然后通过Token变换。输出Y是通过矩阵W与转换后的输入X'之间的乘法得出。

2. **Prefix Tuning：** 这部分说明了Prefix Tuning的过程，其中添加了前缀权重W_p到原始权重W_j中，得到新的权重W'用于生成输出Y。

3. **LoRA调整技术**： 这部分涉及Linear Re-parameterization（线性重新参数化），通过调整矩阵ΔW（通过两个矩阵A和B的乘积表示）来修改权重W。这是一种节省参数调整的方法，使原有模型的W变为W+ΔW，这里也涉及到了一些矩阵运算和优化策略。

其中还提到了两个案例分析的存储需求：“LLAMA”需要65GB，而经过LoRA调整的“QLLoRA”仅需要48GB。

这些笔记对于理解NLP中一些先进的模型调整技术十分有用，尤其对于需要在资源受限的环境下部署NLP模型的研究人员或实践者。

![](./images/handwriting_1.jpg)

In [9]:
content = query_base64_image_description("./images/gpt-4v.jpg")
display(Markdown(content))

NameError: name 'query_base64_image_description' is not defined

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。